In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp

In [ ]:

netgen_mesh = unit_square.GenerateMesh(maxh=1.0)
#help(unit_square)
#netgen_mesh.Refine()
mesh=Mesh(netgen_mesh) # 1.0
#help(mesh)
mesh.Refine()
#mesh.SplitElements_Alfeld()

#mesh.Refine() 
#mesh.Refine() 

print(mesh.ne)

#help(netgen_mesh)
Draw(mesh)

print(mesh)

elvol = Integrate(CoefficientFunction(1),mesh,element_wise=True)
h = [(2*vol)**(1/2) for vol in elvol]
print(min(h),max(h))



96


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

0.1443375667310962 0.1443375678637196


Set correct functionspaces in NGSolve, which will give the corresponding FE Elements, for l=0 its just H1 wihch will give the normal lagrangian elements, which for lowest order are the whitney 0 forms.

In [4]:
#set function space, for l=0 its normal lagrangian finite elements
H_h = Compress(H1(mesh, order = 1)) # main function space
H_H = Compress(H1(mesh, order = 4)) # high order Function space for Riesz representative

L_2 = L2(mesh, order=1)

# print("number of elements in mesh: ", mesh.ne)
# print("number of dofs per element: ", int(H_h.ndof/mesh.ne))

# gfu = GridFunction(H_h,multidim = H_h.ndof)
# for i in range(H_h.ndof):
#     gfu.vecs[i][i] = 0.2
# Draw(gfu, animate = True, min = 0, max = 0.2, order = 2, deformation = True, euler_angles =[-45,-6,25]);



print(H_h.ndof)

25


In [5]:
#choose w_h and set its values on the nodes
w = sin(x**2) + cos(y**2)

w_h = GridFunction(H_h) #set the nodes
w_h.Set(w)

f_H = GridFunction(H_H, name="f_H") #high order 
#w_H.Set(w) #dont need to set the high order gridfunction

#Draw(w_H)

Create the galerkin matrices used to calculate the Riesz representative

In [6]:
#low order
u_h = H_h.TrialFunction() # define the trial function
v_h = H_h.TestFunction() # define the test function

#High order
u_H = H_H.TrialFunction() # define the trial function
v_H= H_H.TestFunction() # define the test function

# m = BilinearForm(H_H) #define m with the L2 inner product on the high order space
# m += u_H*v_H * dx 
# m.Assemble()

m = BilinearForm(H_h,symmetric_storage=True, symmetric=True) #define m with the L2 inner product on the low order space
m += u_h*v_h * dx 
m.Assemble()

a = BilinearForm(H_H, symmetric_storage=True, symmetric=True) # define the H1 inner product on the high order space
a += grad(u_H) * grad(v_H) * dx
a += u_H*v_H * dx 
a.Assemble()


help(BilinearForm)

Help on class BilinearForm in module ngsolve.comp:

class BilinearForm(NGS_Object)
 |  Used to store the left hand side of a PDE. integrators (ngsolve.BFI)
 |  to it to implement your PDE. If the left hand side is linear
 |  you can use BilinearForm.Assemble to assemble it after adding
 |  your integrators. For nonlinear usage use BilinearForm.Apply or
 |  BilinearForm.AssembleLinearization instead of Bilinearform.Assemble.
 |  
 |  Parameters:
 |  
 |  space : ngsolve.FESpace
 |    The finite element space the bilinearform is defined on. This
 |    can be a compound FESpace for a mixed formulation.
 |  
 |  
 |   Keyword arguments can be:
 |  condense: bool = False
 |    (formerly known as 'eliminate_internal')
 |    Set up BilinearForm for static condensation of internal
 |    bubbles. Static condensation has to be done by user,
 |    this enables only the use of the members harmonic_extension,
 |    harmonic_extension_trans and inner_solve. Have a look at the
 |    documentation for

Approximate the dual norm, by the Riesz representive.
Get the Riesz Representive by solving a LGS, since we're approximating we set the space of the Riesz represntative in a higher order space (higher polynomial degree)

In [7]:
#create embedding matrix T

# a_mixed = BilinearForm(trialspace=H_h, testspace=H_H)

# a_dest = BilinearForm(H_H)

# v_H_dual = v_H.Operator("dual")

# dS = dx(element_boundary=True)
# a_dest += u_H*v_H_dual*dx + u_H*v_H_dual*dS
# a_dest.Assemble()

# a_mixed += u_h*v_H_dual*dx + u_h*v_H_dual*dS
# a_mixed.Assemble()

# #build T
# t = a_dest.mat.Inverse() @ a_mixed.mat

############################# doing it directly as a L2 projection

a_mixed = BilinearForm(trialspace=H_h, testspace=H_H)
a_mixed += u_h*v_H*dx
a_mixed.Assemble()

# a_dest = BilinearForm(H_H)
# a_dest += u_H*v_H*dx
# a_dest.Assemble()

#t = a_dest.mat.Inverse() @ a_mixed.mat

#w_H is the high order embedding of w_h
#w_H = GridFunction(H_H, name="w_H")
#w_H.vec.data = t * w_h.vec

#Draw(w_h)
#Draw(w_H)


######################## this works too

#this feels weird, find way to get the range from space H_h itself
#mixed = FESpace([H_h])

#emb = Embedding(H_H.ndof, mixed.Range(0))

#w_H.vec.data = emb * w_h.vec 

#Draw(w_H)

#results in same shape
# print(t.shape)
# print(emb.shape)



In [8]:
help(a.mat.Inverse(freedofs=H_H.FreeDofs()))
a_inv = a.mat.Inverse(freedofs=H_H.FreeDofs())
print(H_h.FreeDofs())
m_inv = m.mat.Inverse(freedofs=H_h.FreeDofs())

#does the same as below
#f_H.vec.data = a_inv @ m.mat * w_H.vec
#Draw(f_H)

#solve for the riesz representative (f_H), wrong redo based on new notes
# temp = f_H.vec.CreateVector()
# temp.data = m.mat * w_H.vec
# f_H.vec.data = a_inv * temp
# Draw(f_H)



Help on BaseMatrix in module ngsolve.la object:

class BaseMatrix(pybind11_builtins.pybind11_object)
 |  Method resolution order:
 |      BaseMatrix
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  AsVector(...)
 |      AsVector(self: ngsolve.la.BaseMatrix) -> ngsolve.la.BaseVector
 |      
 |      Interprets the matrix values as a vector
 |  
 |  CreateColVector(...)
 |      CreateColVector(self: ngsolve.la.BaseMatrix) -> ngsolve.la.BaseVector
 |  
 |  CreateDeviceMatrix(...)
 |      CreateDeviceMatrix(self: ngsolve.la.BaseMatrix) -> BaseMatrix
 |  
 |  CreateMatrix(...)
 |      CreateMatrix(self: ngsolve.la.BaseMatrix) -> BaseMatrix
 |      
 |      Create matrix of same dimension and same sparsestructure
 |  
 |  CreateRowVector(...)
 |      CreateRowVector(self: ngsolve.la.BaseMatrix) -> ngsolve.la.BaseVector
 |  
 |  CreateSparseMatrix(...)
 |      CreateSparseMatrix(self: ngsolve.la.BaseMatrix) -> ngla::BaseSparseMatrix
 | 

then plug the Riesz representive in the GEV problem and solve it, where the largest Eigenvalue will be the constant we're looking for

In [8]:
#create Matrices for GEV problem

#c is the low order galerkin matrix
c = BilinearForm(H_h, symmetric=True, symmetric_storage=True)
c += grad(u_h) * grad(v_h) * dx
c += u_h*v_h * dx 
c.Assemble()

#check norms
# temp3 = c.mat * w_h.vec
# w_h_norm = w_h.vec.InnerProduct(temp3)
# print("H(div) norm of w_h:", sqrt(w_h_norm)) #norm of w_h in H(div,

# temp4 = a.mat * f_H.vec
# f_H_norm = f_H.vec.InnerProduct(temp4)
# print("H(div) norm of f_H, the riesz representative of w_h:", sqrt(f_H_norm)) #norm of the riesz representative

#old b
#b = t.T @ m.mat.T @ a_inv @ m.mat @ t
#new b, uses amixed instead of t
b = c.mat.T @m_inv.T @ a_mixed.mat.T @ a_inv @ a_mixed.mat @ m_inv @ c.mat


C = c.mat.ToDense().NumPy()
# M = m.mat.ToDense().NumPy()
# M_inv = m_inv.ToDense().NumPy()
# A_inv = a_inv.ToDense().NumPy()
# T = a_mixed.mat.ToDense().NumPy()

#M == M.T
# old B 
#B = T.T @ M.T @ A_inv @ M @ T
#new B
B = b.ToDense().NumPy() #C.T @ M_inv.T @ T.T @ A_inv @ T @ M_inv @ C

In [ ]:
#solve GEV using the scipy solver s

#currently still not correct

#y = sp.linalg.eigvals(C,B)
x = sp.sparse.linalg.eigs(C, k=1, M=B, which='SM', return_eigenvectors=False)
#print("maximal Eigenvalue:",max(y))
#print("minimal Eigenvalue:",min(y))
print("minimal Eigenvalue (sparse solver):",x)


h = 1, minEV = 0.76720499
h = 0.5, minEV = 0.76004237
h = 0.25, minEV = 0.59529111
h = 0.125, minEV = 0.52461719 
h = 0.0625, minEV = 0.50758144
h = 0.03125, minEv = 0.50339601
h = 0.015625 , minEV = 0.50235468